# Segmenting Arterial Structure from Radiological Imaging

Tutorial from here: https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

## Import the ArterialVis imaging module

In [ ]:
from arterialvis.download import make_output_dir
from arterialvis.imaging import *

In [ ]:
df = parse_volumes(dicom_path='CTA/bcta1_20171009/SAGHDMIP1')

In [ ]:
patient = load_scan('CTA/bcta1_20171009/SAGHDMIP1')

In [ ]:
patient

## Extract the pixels from the DICOM image files

In [ ]:
imgs = get_pixels_hu(patient)

In [ ]:
np.save("DCM_array.npy", imgs)

In [ ]:
file_used="DCM_array.npy"
imgs_to_process = np.load(file_used).astype(np.float64)
flat = imgs_to_process.flatten()

In [ ]:
plt.hist(flat, bins=50, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
id = 0
imgs_to_process = np.load("DCM_array.npy")

def sample_stack(stack, rows=6, cols=6, start_with=10, show_every=3):
    fig,ax = plt.subplots(rows,cols,figsize=[12,12])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

sample_stack(imgs_to_process)

In [ ]:
print("Slice Thickness: %f" % patient[0].SliceThickness)

In [ ]:
patient[0].PixelSpacing

In [ ]:
import scipy

In [ ]:
print("Pixel Spacing (row, col): (%f, %f) " % (patient[0].PixelSpacing[0], patient[0].PixelSpacing[1]))

In [ ]:
id = 0
imgs_to_process = np.load("DCM_array.npy")
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = map(float, ([scan[0].SliceThickness, scan[0].PixelSpacing[0], scan[0].PixelSpacing[1]]))
    spacing = np.array(list(spacing))

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

print("Shape before resampling\t", imgs_to_process.shape)
imgs_after_resamp, spacing = resample(imgs_to_process, patient, [1,1,1])
print("Shape after resampling\t", imgs_after_resamp.shape)

In [ ]:
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from plotly.tools import FigureFactory as FF
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
def make_mesh(image, threshold=-400, step_size=1):

    print("Transposing surface")
    p = image.transpose(2,1,0)
    
    print("Calculating surface")
    verts, faces, norm, val = measure.marching_cubes(p, threshold, step_size=step_size, allow_degenerate=True) 
    return verts, faces, norm, val

def plotly_3d(verts, faces):
    x,y,z = zip(*verts) 
    
    print("Drawing")
    
    # Make the colormap single color since the axes are positional not intensity. 
#    colormap=['rgb(255,105,180)','rgb(255,255,51)','rgb(0,191,255)']
    colormap=['rgb(139,0,0)','rgb(255,0,0)']
    
    fig = FF.create_trisurf(x=x,
                        y=y, 
                        z=z, 
                        plot_edges=False,
                        colormap=colormap,
                        simplices=faces,
                        backgroundcolor='rgb(255, 255, 255)',
                        title="Interactive Visualization")
    iplot(fig)
    return fig

def plt_3d(verts, faces):
    print("Drawing")
    x,y,z = zip(*verts) 
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], linewidths=0.05, alpha=1)
    face_color = [1, 1, 0.9]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, max(x))
    ax.set_ylim(0, max(y))
    ax.set_zlim(0, max(z))
    ax.set_facecolor((0.7, 0.7, 0.7))
    plt.show()
    return plt

In [ ]:
v, f, norm, val = make_mesh(imgs_after_resamp, 110)
plt_3d(v, f)

In [ ]:
(zip(v))

In [ ]:
norm

In [ ]:
 pyfig= plotly_3d(v, f)

In [ ]:
pyfig.write_html('DCM_plotlyseg_110.html')